# Module 11 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID. Make sure you submit both an .ipynb and .html version of your *completed* notebook. You can generate the HTML version using:

> ipython nbconvert [notebookname].ipynb

or use the File menu.

Add whatever additional imports you require here. Stick with the standard libraries and those required by the class. The import
gives you access to these functions: http://ipython.org/ipython-doc/stable/api/generated/IPython.core.display.html (Copy this link)
Which, among other things, will permit you to display HTML as the result of evaluated code (see HTML() or display_html()).

In [1]:
from __future__ import division # so that 1/2 = 0.5 and not 0
from IPython.core.display import *
import numpy as np
import csv

## Decision Trees

For this assignment you will be implementing and evaluating a Decision Tree using the ID3 Algorithm (**no** pruning or normalized information gain). Use the provided pseudocode. The data is located at (copy link):

http://archive.ics.uci.edu/ml/datasets/Mushroom

You can download the two files and read them to find out the attributes, attribute values and class labels as well as their locations in the file.

One of the things we did not talk about in the lectures was how to deal with missing values. In C4.5, missing values were handled by treating "?" as an implicit attribute value for every feature. For example, if the attribute was "size" then the domain would be ["small", "medium", "large", "?"]. Another approach is to skip instances with missing values. Yet another approach is to infer the missing value conditioned on the class. For example, if the class is "safe" and the color is missing, then we would infer the attribute value that is most often associated with "safe", perhaps "red". **Use the "?" approach for this assignment.**

As we did with the neural network, you should randomize your data (always randomize your data...you don't know if it is in some particular order like date of collection, by class label, etc.) and split it into two (2) sets. Train on the first set then test on the second set. Then train on the second set and test on the first set.

For regression, we almost always use something like Mean Squared Error to judge the performance of a model. For classification, there are a lot more options but for this assignment we will just look at classification error:

$$error\_rate=\frac{errors}{n}$$

You must implement four functions. `train` takes training_data and returns the Decision Tree as a data structure or object (for this one, I'm removing the OOP restriction...people often feel more comfortable writing a Tree in an OOP fashion). Make sure your Tree can be represented somehow.

```
def train( training_data):
   # returns a decision tree data structure
```

and `view` takes a tree and prints it out:

```
def view( tree):
    pass # probably doesn't return anything.
```

the purpose of the function is to be able to see what the tree looks like. It should be legible/pretty. You can use ASCII if you like or use something like NetworkX.

and `classify` takes a tree and a List of instances (possibly just one) and returns the classifications:

```
def classify( tree, test_data):
    # returns a list of classifications
```

and `evaluate` takes the classifications and the test_data and returns the error rate:

```
def evaluate( test_data, classifications):
    # returns an error rate
```

Basically, you're going to:

1. learn the tree for set 1
2. view the tree
3. classify set 2
4. evaluate the tree
5. learn the tree for set 2
6. view the tree
7. classify set 1
8. evalute the tree
9. average the classification error.

This is all that is required for this assignment. I'm leaving more of the particulars up to you but you can definitely use the last module as a guide.

**This is a very important assignment to reflect on the use of deepcopy because it has a natural recursive implementation**

-----

### Decision Tree class ###  
The decision tree is implemented as `DTnode` as a tree with root at a ndoe. The class contains three fields: `attrib`, `isLeaf`, and `children`. The first is the attribute which the decision tree is split on. The second denotes whether the node is a leaf node. For a leaf node, the attribut becomes the classification. The `children` is a `dict` containing the child nodes keyed by the values of the attribute.

To instantiate a `DTnode`, one passes a string argument as the attribute for which the node is supposed to represent. 

The **addChild(nd, val)** function adds a child node `nd` with the `val` of the attribute.

The **getChild(val)** function returns the child node for `val` value of attribute.

The **getValues()** function returns set of all possible values for the attribute.


In [2]:
class DTnode:
    def __init__(self, attrib):
        self.attrib = attrib
        self.isLeaf = True
        self.children = dict()

    def addChild(self, node, val):
        self.isLeaf = False
        self.children[val] = node
        
    def getChild(self, val):
        return self.children[val]
    
    def getValues(self):
        return self.children.keys()

    def __str__(self):
        if self.isLeaf:
            childTxt = 'terminal'
        else:
            childTxt = 'child: ' + str(list(self.children.keys()))
        return '[Node for %s, %s ]'%(self.attrib, childTxt)

---

### Reading in data ###

The function **readCSV(filePath** reads a CSV file and returns the content of the file as a list of lists of strings.

The header of the data is encoded below, which are used to 1) produce a set of attributes for the training of the decision tree, and 2) mark the fields of the numpy recarray which are used to contain the data and make programming easier.

The rows of data are shuffled and split into two data sets.

In [3]:
def readCSV(filePath):
    with open(filePath, 'rt') as f:
        reader = csv.reader(f)
        l = list(reader)
    return l

headers = ['class', 'cap-shape', 'cap-surface', 'cap-color' , 'bruises', 'odor', 
           'gill-attachment' , 'gill-spacing', 'gill-size', 'gill-color' , 'stalk-shape',
           'stalk-root', 'stalk-surface-above-ring' , 'stalk-surface-below-ring',
           'stalk-color-above-ring', 'stalk-color-below-ring' , 'veil-type', 'veil-color',
           'ring-number' , 'ring-type', 'spore-print-color', 'population', 'habitat']
attribSet = set(headers) - set(['class'])

raw = np.rec.fromrecords( readCSV('agaricus-lepiota.data') , names=headers)
np.random.shuffle(raw) # shuffle the data
data1 = raw[:np.floor(raw.size/2)] # first half of data
data2 = raw[np.floor(raw.size/2):] # second half of data

### Helper functions ###

**B(p)**  
Entropy function for binary variables. Returns:
$$B(q) = -[q \lg({q}) + (1-q) \lg({1-q})]$$
Where $q$ is the proportion of data taking one of the binary values.

**edibleProb(d)**  
The function calculates the proportion of the data `d` which are within the class of edible mushrooms.

** Remainer(d, Attr)**  
For a specific attribute `Attr` with d distinct values divides the training set E into subsets $E_1, ... , E_d$. Each subset $E_k$ has $p_k$ positive examples and $n_k$ negative examples. The function calculates the expected entropy of using `Attr` as the dividing attribute for the data `d`. Formally, it calculates:
$$\sum_{k=1}^{d}{\frac{p_k+n_k}{p+n}B(\frac{p_k}{p_k+n_k})}$$

** Importance(d, attribs)**  
Given a data set as a recarray `d` and a set of attributes for which to divide the data with, the function returns a tuple containing the most important attribute by information gain and the information gain. Formally, it calcualtes

$$argmax_{A\in attributes} Gain(A)$$
where
$$Gain(A) = B(\frac{p}{p+n}) - Remainer(A)$$

In [4]:
def B(p):
    return 0 if p==1 or p==0 else -(p*np.log2(p) + (1-p)*np.log2(1-p))

def edibleProb(d):
    return np.count_nonzero(d['class']=='e') / d.size
    
def Remainder(d, Attr):
    vals,N = np.unique(d[Attr], return_counts=True )
    entropies = [B(edibleProb(d[d[Attr]==v])) for v in vals]
    return np.sum(N/d.size * entropies)

def Importance(d, attribs):
    entropy = B(edibleProb(d))
    gains = list(zip(*[(entropy - Remainder(d,A),A) for A in attribs]))
    return gains[1][np.argmax(gains[0])], np.max(gains[0])

The training function implements the id3 algorithm by recursively picking attributes of the data which gives the most information gain. For each value of the attribute, a child node is added to the tree and the data is subsetted which are homogenous on the given attribute. The algorithm stops when all data are of the same class, the data subset is empty, or there are no more attributes to split the data on. The the latter two, the algorithm chooses the majority class.

In [5]:
def train(data):
    def id3(d, attribs, default):
        if d.size == 0: # empty data
            return DTnode(default)
        prEd = edibleProb(d)
        majority = 'e' if prEd > 0.5 else 'p'
        if prEd in [0,1] or len(attribs)==0: # homogenous or no attribs
            return DTnode(majority) 

        bestAttr, gain = Importance(d, attribs)
        attribSubset = attribs - set([bestAttr])
        nd = DTnode(bestAttr) # new node at the best attribute
        for v in set(d[bestAttr]):
            child = id3(d[d[bestAttr]==v], attribSubset, majority)
            nd.addChild(child, v)
        return nd

    return id3(data, attribSet, 'e' if edibleProb(data) > 0.5 else 'p')

The **view(tree)** function printes an ASCII tree where each child node is printed at an additional indent level of its parent. For leaf node, the tree is printed with the class. The algorithm is recursive and prints the tree via in-order traversal. The tree looks like:
````
Attribute [var1]:
	value = a, class: p
    value = b, Attribute [var2]:
    	value = c, class: p
		value = d, class: n
	value = e, class = n
````

In [6]:
def view(tree):
    def toStr(nd, level=0):
        if nd.isLeaf:
            return 'class: %s\n' % nd.attrib
        else:
            ret = 'Attribute [' + nd.attrib + "]:\n"
            nx = level + 1
            for key in nd.children:
                ret += "\t"*nx + 'value = %s, '%key \
                    + toStr(nd.children[key],nx)
            return ret
    
    print(toStr(tree))

The **class(tree, data)** function uses the decision tree `tree` to classifiy `data` and returning the classification as a numpy array. The algorithm performs in-order traversal of the decision tree through recursion and mark the appropriate subset data with the correct class.

In [7]:
def classify(tree, data):
    def recurClass(d, nd, idx, res):
        if nd.isLeaf:
            res[idx] = nd.attrib
            return out
        else:
            for k in nd.getValues():
                recurClass(d, nd.getChild(k), \
                           np.logical_and(idx, d[nd.attrib]==k), res)
            return out
        
    ind = np.array([True] * data.size)
    out = np.array([None] * data.size)
    return recurClass(data, tree, ind, out)

The **evaulate(data, classes)** function calculates the error rate of the classification, or formally:

$$\frac{1}{n}\sum_{i}^{n}{I_{k_i\ne\hat{k_i}}(i\in x)}$$

In [8]:
def evaluate(data, classes):
    return sum(data['class'] != classes) / data.size

---
### Training Decision Trees and Evaulation###

** Train Decision Tree 1 from set 1**

In [9]:
tree1 = train(data1)

In [10]:
view(tree1)

Attribute [odor]:
	value = a, class: e
	value = n, Attribute [spore-print-color]:
		value = h, class: e
		value = w, Attribute [habitat]:
			value = w, class: e
			value = g, class: e
			value = d, Attribute [stalk-root]:
				value = b, class: e
				value = ?, class: p
			value = l, Attribute [stalk-color-below-ring]:
				value = w, class: p
				value = n, class: e
				value = y, class: p
			value = p, class: e
		value = n, class: e
		value = k, class: e
		value = y, class: e
		value = o, class: e
		value = r, class: p
		value = b, class: e
	value = f, class: p
	value = p, class: p
	value = y, class: p
	value = s, class: p
	value = m, class: p
	value = c, class: p
	value = l, class: e



**Classify set 2 with tree 1 **

In [11]:
k2 = classify(tree1, data2)

** Train Decision Tree 2 from set 2**

In [12]:
tree2 = train(data2)

In [13]:
view(tree2)

Attribute [odor]:
	value = a, class: e
	value = n, Attribute [spore-print-color]:
		value = h, class: e
		value = w, Attribute [stalk-surface-below-ring]:
			value = s, Attribute [cap-surface]:
				value = y, class: e
				value = g, class: p
				value = f, class: e
				value = s, class: e
			value = f, class: e
			value = y, Attribute [stalk-root]:
				value = ?, class: p
				value = b, class: e
				value = c, class: p
			value = k, class: e
		value = n, class: e
		value = k, class: e
		value = y, class: e
		value = o, class: e
		value = r, class: p
		value = b, class: e
	value = f, class: p
	value = p, class: p
	value = y, class: p
	value = s, class: p
	value = m, class: p
	value = c, class: p
	value = l, class: e



**Classify set 1 with tree 2 **

In [14]:
k1 = classify(tree2, data1)

---
**Evaulate average error rate**

In [15]:
e = evaluate(np.hstack((data1,data2)), np.hstack((k1,k2)))*100
print('Average error rate: %.2f%%' % e)

Average error rate: 0.05%
